# Cap 3. - Dicionários e Conjuntos
### "Mapeamentos com consulta de chave flexível" até "variações de dict" (pag 101 a 105)

```
Data: 19/08/2020
Páginas: 100 a 106
Livro/Edição: Python Fluent, Ed.1
Python: 3.8.2
```

---

### Tópicos
- Default dict
    - Como usar
    - Particularidades e pontos de atenção
- Método `__missing__`
    - Usando para implementação customizada

## 1. Default dict

"Subclasse de dict que chama uma _factory_ para fornecer valores faltantes." - Python docs

Ao instanciar um defaultdict, você fornece uma função ou classe (que será armazenada no **atributo** _default_factory_) que será usda para **gerar um valor sempre que tentarem acessar diretamente uma chave inexistente no dicionário**.

Mais em:
- [Collections.defaultdict python3](https://docs.python.org/3/library/collections.html#collections.defaultdict) <- indico muito dar uma olhada
- Bônus: [refactoring.guru - padrão de design _factory_](https://refactoring.guru/pt-br/design-patterns/factory-method)

In [115]:
# PARA COMPARAÇÃO - dicionário normal

dict_padrao = {'a': 2}

# acessar uma chave existente retorna seu valor
x = dict_padrao['a']
print(x)  # R: 2

# acessar uma chave inexistente com get retorna o valor definido como default
# Nota: o dicionário original não é alterado
x = dict_padrao.get('b', 'teste')
print(x)  # R: teste

# tentativa de acessar diretamente chave inexistente
dict_padrao['c'] # raise KeyError: 'c'

2
teste


KeyError: 'c'

In [117]:
# USANDO defaultdict
from collections import defaultdict

def_dict = defaultdict(int) # -> sempre que uma missing key for acessada,
                            #    essa função default_factory() será chamada p/ repor o valor
                            #    por default, default_factory = None
def_dict['a'] = 2

# acessar uma chave existente retorna seu valor
# igual dict normal
x = def_dict['a']
print(x)  # R: 2

# acessar uma chave inexistente com get retorna o VALOR DEFAULT DE GET, NÃO USA O DEFAULT FACTORY !!
# Nota: o dicionário original NÃO é alterado !!
# igual dict normal
x = dict_padrao.get('b', 'teste')
print(x)  # R: teste

# tentativa de acessar diretamente chave inexistente
# Nota: o dicionário original É alterado !!
x = def_dict['c']
print(x) # R: 0 -> int()
def_dict

2
teste
0


defaultdict(int, {'a': 2, 'c': 0})

In [118]:
# default_dict sem factory é igual a um dict normal e gera KeyError para chaves ausentes
def_dict = defaultdict()
def_dict['a']

KeyError: 'a'

In [53]:
# Caso de uso prático -> inicializar listas quando não existem para dar append

lista_de_pedidos = [('Ana', 'boneca'), ('Ana', 'ursinho'), ('Bob', 'carrinho'), ('Joe', 'livro'), ('Bob', 'ursinho')]
presentes_por_criança = defaultdict(list)

for nome, presente in lista_de_pedidos:
    presentes_por_criança[nome].append(presente)

presentes_por_criança

defaultdict(list,
            {'Ana': ['boneca', 'ursinho'],
             'Bob': ['carrinho', 'ursinho'],
             'Joe': ['livro']})

### 1.1. Observações e pontos de atenção

In [59]:
# default_factory é um ATRIBUTO, não um ARGUMENTO/PARÂMETRO
def_dict = defaultdict(default_factory=int)
display(def_dict)  # -> perceba que defaultdict identifica default_factory como uma CHAVE com o VALOR int

def_dict['a']  # raise KeyError

defaultdict(None, {'default_factory': int})

KeyError: 'a'

In [75]:
# FORMAS DE PASSAR VALORES PARA UM DEFAULT DICT

# via parâmetros
def_dict = defaultdict(default_factory=int, a=2, b='a')  # PS: não funciona para chaves que são números
display(def_dict)

# via dicionário
def_dict = defaultdict(None, {'a': 2, 'b':3})
display(def_dict)

# ambos (parâmetros e dicionários)
def_dict = defaultdict(None, {'a': 2, 'b':3}, batata=2)
display(def_dict)

# com um iterável
def_dict = defaultdict(list, [('a', 2), (1, 3), ('batata', 4)])
display(def_dict)

# NÃO FUNCIONA
# def_dict = defaultdict({'a': 2, 'b':3})  # Erro porque ele não identifica a factory como callable
# def_dict = defaultdict(None, {'a': 2}, {'b':3})  # Erro porque só é esperado 1 argumento para popular o dict

defaultdict(None, {'default_factory': int, 'a': 2, 'b': 'a'})

defaultdict(None, {'a': 2, 'b': 3})

defaultdict(None, {'a': 2, 'b': 3, 'batata': 2})

defaultdict(list, {'a': 2, 1: 3, 'batata': 4})

In [82]:
# FORMAS DE DEFINIR A default_factory PARA UM DEFAULT DICT

# Passando uma função (ou não passando nada)
def minha_factory():  # -> não recebe nada por parâmetro
    return 'valor'
def_dict = defaultdict(minha_factory)
display(def_dict)

# Via lambda
def_dict = defaultdict(lambda: 'Oh yeas!')
display(def_dict)


# Atribuindo uma função diretamente a default_factory
def_dict = defaultdict(list, {'teste': 'pela ciência'})
display(def_dict)
def_dict.default_factory = int
display(def_dict)

defaultdict(<function __main__.minha_factory()>, {})

defaultdict(<function __main__.<lambda>()>, {})

defaultdict(list, {'teste': 'pela ciência'})

defaultdict(int, {'teste': 'pela ciência'})

In [98]:
# TESTE COM FUNÇÃO GERADORA

class ClasseGeradora:
    max_numeros = 100
    gerador_de_numeros = (i for i in range(max_numeros))
    def prox_valor(self):
        return next(self.gerador_de_numeros)

c = ClasseGeradora()
def_dict = defaultdict(c.prox_valor)

# Para cada chamada a uma chave inexistente nova um novo valor será retornado
# OBS: Se for necessário criar mais números que o que doi definido em
#      "max_numeros" a função "prox_valor" irá quebrar!
print("def_dict['a']: ", def_dict['a'])
print("def_dict['b']: ", def_dict['b'])
print("def_dict['c']: ", def_dict['c'])
display(def_dict)

def_dict['a']:  0
def_dict['b']:  1
def_dict['c']:  2


defaultdict(<bound method ClasseGeradora.prox_valor of <__main__.ClasseGeradora object at 0x110856eb0>>,
            {'a': 0, 'b': 1, 'c': 2})

In [105]:
# Exemplo mostrando uma factory e também mostrando
# para ter cuidado com as referências dos objetos
import collections

"""
Suponha que somos um ecomerce e guardamos todas
as sessões ativas em um dicionário.

Mesmo que o usuário não tenha efetuado o login,
nós guardamos seu carrinho de compras para que 
ele não seja perdido quando o login for efetuado
"""

# Essa é nossa classe de usuário
class Usuario:
    def __init__(self, nome, carrinho):
        self.nome = nome
        self.carrinho = carrinho
    def __repr__(self):
        return f'Usuario {id(self)} - nome: {self.nome} | carrinho: {self.carrinho}'

class FabricaDeUsuarios:
    nome_default = 'desconhecido'
    carrinho_compras_default = []  # -> aqui está o problema, variáveis no escopo da classe mantém a mesma 
                                   #    referência para todos os objs construídos a partir delas
        
    def novo_usuario_desconhecido(self):
        # Todos os objs construídos vão referenciar a mesma lista no seu carrinho de compras...
        novo_usuario = Usuario(nome=self.nome_default, carrinho=self.carrinho_compras_default) 
        return novo_usuario
    
    def novo_usuario_cadastrado(self, nome, carrinho):  # Só para exemplificar métodos de uma factory
        ...
    
# Podemos usar default dict para automaticamente criar um novo usuario
# desconhecido quando não temos um usuario logado para a sessão
factory = FabricaDeUsuarios()
sessoes_ativas = defaultdict(factory.novo_usuario_desconhecido)

print('CRIANDO USUARIOS AUTOMATICAMENTE QUANDO NÃO ENCONTRADOS...')
usuario1 = sessoes_ativas['ip_usuario1']
usuario2 = sessoes_ativas['ip_usuario2']
display(usuario1)
display(usuario2)

print('-' * 60)
print('MUDAR O CARRINHO DE UM USUÁRIO MUDA O CARRINHO DE TODOS OS')
print('OUTROS POR CAUSA DA REF. NO OBJETO...')
usuario2.carrinho += ['banana']
display(usuario1)
display(usuario2)

CRIANDO USUARIOS AUTOMATICAMENTE QUANDO NÃO ENCONTRADOS...


Usuario 4571934048 - nome: desconhecido | carrinho: []

Usuario 4571933424 - nome: desconhecido | carrinho: []

------------------------------------------------------------
MUDAR O CARRINHO DE UM USUÁRIO MUDA O CARRINHO DE TODOS OS
OUTROS POR CAUSA DA REF. NO OBJETO...


Usuario 4571934048 - nome: desconhecido | carrinho: ['banana']

Usuario 4571933424 - nome: desconhecido | carrinho: ['banana']

## 2. Método `__missing__`

Livro: "Esse método **não está definido na classe base _dict_**, porém `dict` está ciente de sua finalidade, se você criar uma subclasse de `dict` e implementar o método `__missing__`, o método `dict.__getitem__` padrão o chamará sempre que uma chave não for encontrada, em vez de gerar um `KeyError`"

**PS:** "A presença de um método `__missing__` não terá nenhum efeito no comportamento de outros métodos que consultem chaves, por exemplo `get` ou `__contains__` (que implementa o operador `in`).

- Não encontrei a implementação oficial de python de um dict para ver como esse método se relaciona com a classe :( 
- Encontrei essa implementação um gist (NÃO OFICIAL) que é interessante para ver como ele usa os _dunder methods_ para implementar um defaultdict customizado https://gist.github.com/ohe/1605376


In [120]:
# EXEMPLO DO LIVRO
# MANEIRA DE FAZER UMA IMPLEMENTAÇÃO CUSTOMIZADA

"""
Objetivo: dicionário que permite encontrar o valor tanto usando
a representação da chave como string quanto como inteiro

PS: Garante a consistência na BUSCA, não na INSERÇÃO
"""
class StrKeyDict0(dict): # -> A classe herda de dict
    
    def __missing__(self, key):
        if isinstance(key, str):  # importante para não gerar uma recursão infinita (prox. comment)
            raise KeyError(key)
        return self[str(key)]     # aqui chamamos __getitem__ que se falhar chama __missing__ ... (!!)
    
    def get(self, key, default=None):
        try:
            return self[key]  # nesse caso, fizemos get delegar para __getitem__
                              # dando a oportunidade do nosso __missing__ agir
        except KeyError:
            return default    # como __missing__ falhou, devolvemos default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() # OBS: não podemos buscar a chave de maneira
                                                             # pythonica convencional usando: key in self
                                                             # porque se não __contains__ seria chamado
                                                             # recursivamente, por isso evitamos isso
                                                             # chamando explicitamente self.keys()
    

# TESTES
custom_dict = StrKeyDict0({'2': 'two', '4': 'four'})
display(custom_dict)
print(custom_dict['4'])
print(custom_dict[4])
print(custom_dict[3]) # como esperado, gera KeyError porque '3' não existe no dicionario

{'2': 'two', '4': 'four'}

four
four


KeyError: '3'